In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pycaret
from pycaret.time_series import *
import wbgapi as wb

In [2]:
pais=['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
pais_lista=[]
ev_2020_lista=[]
ev_2025_lista=[]
ev_2025_mejora_lista=[]

In [3]:
for i in pais:    
    data = wb.data.DataFrame(['SP.DYN.LE00.IN','SP.DYN.IMRT.IN'], i, columns='time',time= range(1960,2021), numericTimeKeys=True)
    pais_lista.append(i)
    
    data =data.transpose()
    data['EV_Total']=data['SP.DYN.LE00.IN']
    data['Mortalidad_Infantil']=data['SP.DYN.IMRT.IN']
    data.drop(['SP.DYN.LE00.IN','SP.DYN.IMRT.IN'], axis = 1,inplace=True)
    data2 = data.copy()
    data['date']=data.index
    data['date'] = pd.to_datetime(data['date'],format='%Y')
    data=data.set_index('date')
    ev_2020_lista.append(data['EV_Total'].tail(1).values)
    metric = "mase"
    exclude = ["auto_arima", "bats", "tbats", "lar_cds_dt", "par_cds_dt"]

    s = setup(data,target='EV_Total', fh = 10,
                enforce_exogenous = False, numeric_imputation_exogenous="linear", numeric_imputation_target= "linear", verbose=False) 
                
    best =  s.compare_models(n_select=1, turbo=True,verbose=False, exclude = ["omp_cds_dt"])

    final_best1 = finalize_model(best)

    df_fut1=predict_model(best, fh = 15)
    ev_2025_lista.append(df_fut1['y_pred'].tail(1).values)

    i = 1
    year2 = (2020 + i)
    while i < 5:
        #metric = "mase"
        #exclude = ["auto_arima", "bats", "tbats", "lar_cds_dt", "par_cds_dt"]
        s = setup(data,target='EV_Total', fh = 10,
                    enforce_exogenous = False, numeric_imputation_exogenous="linear", numeric_imputation_target= "linear", verbose=False) 
        best =  s.compare_models(n_select=1, turbo=True,verbose=False, exclude = ["omp_cds_dt"])#, exclude = ["rf_cds_dt"]

        final_best = finalize_model(best)

        df_fut=predict_model(best, fh = 11)

        new_mort=(data.loc[data.index[-1], "Mortalidad_Infantil"])*0.9
        new_ev = df_fut.loc[df_fut.index[-1], "y_pred"]

        new_row = {'index':year2, 'EV_Total':new_ev, 'Mortalidad_Infantil':new_mort}
        #data2=(data.tail(1).index[0]+pd.Timedelta(days=366))
        data2=(data.tail(1).index[0]+pd.offsets.DateOffset(years=1))
        data.loc[data2]=new_row

        i+=1

    s = setup(data,target='EV_Total', fh = 10,
                enforce_exogenous = False, numeric_imputation_exogenous="linear", numeric_imputation_target= "linear", verbose=False) 
    best =  s.compare_models(n_select=1, turbo=True,verbose=False, exclude = ["omp_cds_dt"])

    final_best = finalize_model(best)

    df_fut2=predict_model(best, fh = 11)
    ev_2025_mejora_lista.append(df_fut2['y_pred'].tail(1).values)
    

In [15]:
resultados = pd.DataFrame(pais_lista, columns=['Pais'])
resultados1 = pd.DataFrame(ev_2020_lista, columns=['EV_2020'])
resultados2 = pd.DataFrame(ev_2025_lista, columns=['EV_2025'])
resultados3 = pd.DataFrame(ev_2025_mejora_lista, columns=['EV_2025_Mejora'])
resultado = pd.concat([resultados,resultados1,resultados2,resultados3],axis=1)
resultado.head()